In [1]:
! pip install xgboost

     --------------------------------------- 99.8/99.8 MB 19.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.metrics import accuracy_score,confusion_matrix,roc_curve,roc_auc_score,classification_report,r2_score,mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns 


import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [16]:
import xgboost as xgb

In [17]:
df =  pd.read_csv(r"vehicles_data_students.csv")
df.head()


,Unnamed: 0,id,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,county,state,lat,long
0,55483,7315914053,0,2018.0,ram,promaster 2500,excellent,NaN,gas,44244.0,clean,automatic,NaN,NaN,van,NaN,NaN,ca,32.792800,-116.966500
1,162368,7310885048,13995,2017.0,mazda,cx-3,NaN,4 cylinders,gas,7037.0,rebuilt,automatic,NaN,NaN,SUV,white,NaN,ia,41.207382,-96.023096
2,234393,7308243856,19990,2019.0,mitsubishi,eclipse cross sp,good,NaN,gas,35313.0,clean,other,4wd,NaN,hatchback,white,NaN,nc,35.190000,-80.830000
3,276110,7315817729,0,2019.0,honda,cr-v,NaN,NaN,gas,25626.0,clean,automatic,NaN,NaN,SUV,orange,NaN,ny,40.854573,-74.120219
4,349033,7301620999,42900,2015.0,chevrolet,corvette,excellent,8 cylinders,gas,29000.0,clean,automatic,NaN,NaN,convertible,black,NaN,sc,34.755562,-82.906419


In [18]:
list(df.columns)

['Unnamed: 0',
 'id',
 'price',
 'year',
 'manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'drive',
 'size',
 'type',
 'paint_color',
 'county',
 'state',
 'lat',
 'long']

In [19]:
drop_columns = ['Unnamed: 0',
 'id','lat',
 'long','title_status','county']
df = df.drop(columns=drop_columns)

In [20]:
df

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,state
0,0,2018.0,ram,promaster 2500,excellent,NaN,gas,44244.0,automatic,NaN,NaN,van,NaN,ca
1,13995,2017.0,mazda,cx-3,NaN,4 cylinders,gas,7037.0,automatic,NaN,NaN,SUV,white,ia
2,19990,2019.0,mitsubishi,eclipse cross sp,good,NaN,gas,35313.0,other,4wd,NaN,hatchback,white,nc
3,0,2019.0,honda,cr-v,NaN,NaN,gas,25626.0,automatic,NaN,NaN,SUV,orange,ny
4,42900,2015.0,chevrolet,corvette,excellent,8 cylinders,gas,29000.0,automatic,NaN,NaN,convertible,black,sc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64027,3495,1997.0,mercury,grand marquis,good,8 cylinders,gas,106253.0,automatic,rwd,NaN,sedan,white,nj
64028,25590,2018.0,lincoln,mkz reserve hybrid sedan,good,NaN,hybrid,33467.0,other,fwd,NaN,sedan,white,mn
64029,22421,2015.0,mercedes-benz,e-class,excellent,NaN,gas,55494.0,automatic,rwd,NaN,sedan,black,or
64030,29999,2013.0,jeep,wrangler unlimited sport,NaN,NaN,gas,63408.0,automatic,4wd,NaN,SUV,white,id


In [21]:
df.shape

(64032, 14)

In [22]:
df.isna().sum()

price               0
year              158
manufacturer     2569
model             802
condition       26097
cylinders       26511
fuel              424
odometer          669
transmission      353
drive           19471
size            45841
type            13785
paint_color     19505
state               0
dtype: int64

In [23]:
df = df.dropna()
df.head(5)

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,state
5,0,2006.0,chrysler,300,like new,8 cylinders,gas,149000.0,automatic,rwd,full-size,sedan,white,fl
9,20995,2011.0,chevrolet,silverado 1500,excellent,8 cylinders,gas,92001.0,automatic,4wd,mid-size,truck,blue,wi
30,13000,2012.0,infiniti,g37x,good,6 cylinders,gas,49200.0,automatic,4wd,mid-size,sedan,black,oh
32,12500,2017.0,buick,encore premium sport8,like new,4 cylinders,gas,73057.0,automatic,4wd,mid-size,SUV,black,ny
39,44500,2015.0,gmc,yukon xl denali 4x4,excellent,8 cylinders,gas,91000.0,automatic,4wd,full-size,SUV,white,az


In [24]:
df.shape

(11896, 14)

In [25]:
df.describe()

,price,year,odometer
count,1.189600e+04,11896.000000,1.189600e+04
mean,1.452767e+04,2008.464778,1.258991e+05
std,1.566235e+05,10.026049,2.561703e+05
min,0.000000e+00,1918.000000,0.000000e+00
25%,4.988000e+03,2006.000000,7.336900e+04
50%,9.000000e+03,2011.000000,1.141480e+05
75%,1.700000e+04,2014.000000,1.560960e+05
max,1.700000e+07,2021.000000,1.000000e+07


In [26]:
df.drop_duplicates(inplace=True)

In [27]:
df.shape

(11323, 14)

## FILTER CATEGORICAL FEATURES

In [28]:
object_type = list(df.select_dtypes(include='object'))
object_type

['manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'transmission',
 'drive',
 'size',
 'type',
 'paint_color',
 'state']

## ENCODING

In [29]:
df_dum = pd.get_dummies(df[object_type],drop_first=True)

In [30]:
df_dum.head()

,manufacturer_alfa-romeo,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,manufacturer_chrysler,manufacturer_datsun,manufacturer_dodge,manufacturer_ferrari,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
5,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
30,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
32,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
39,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [31]:
df_dum.shape

(11323, 3439)

In [32]:
df = df.join(df_dum)

In [33]:
df.shape

(11323, 3453)

In [34]:
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
5,0,2006.0,chrysler,300,like new,8 cylinders,gas,149000.0,automatic,rwd,...,False,False,False,False,False,False,False,False,False,False
9,20995,2011.0,chevrolet,silverado 1500,excellent,8 cylinders,gas,92001.0,automatic,4wd,...,False,False,False,False,False,False,False,True,False,False
30,13000,2012.0,infiniti,g37x,good,6 cylinders,gas,49200.0,automatic,4wd,...,False,False,False,False,False,False,False,False,False,False
32,12500,2017.0,buick,encore premium sport8,like new,4 cylinders,gas,73057.0,automatic,4wd,...,False,False,False,False,False,False,False,False,False,False
39,44500,2015.0,gmc,yukon xl denali 4x4,excellent,8 cylinders,gas,91000.0,automatic,4wd,...,False,False,False,False,False,False,False,False,False,False


In [35]:
df.drop(columns = object_type,inplace=True)

In [36]:
df.head(2)

,price,year,odometer,manufacturer_alfa-romeo,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,manufacturer_chrysler,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
5,0,2006.0,149000.0,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
9,20995,2011.0,92001.0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,True,False,False


#### selecting price range 

In [37]:
df = df[df['price']>1000]
df = df[df['price']<25000]

In [38]:
df.shape

(9009, 3442)

In [39]:
y = df['price']
X = df.drop(['price'],axis = 1)

In [40]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 10)

# XGB

In [41]:
import xgboost as xgb
xgb = xgb.XGBRegressor()
xgb.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [42]:
y_pred=xgb.predict(X_test)

In [43]:
r2_score(y_test,y_pred)

0.7172541492575935